In [1]:
import sys
import tensorflow as tf
sys.path.append('../../generative-adversarial-nets/')
from ganetwork import GAN, CGAN
from sklearn import datasets
from sklearn.preprocessing import minmax_scale, LabelBinarizer

In [2]:
ds = datasets.load_iris()
X, y = minmax_scale(ds.data), LabelBinarizer().fit_transform(ds.target)

In [3]:
discriminator_layers=[(4, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(6, None), (5, tf.nn.tanh), (4, None)]
gan = GAN(discriminator_layers, generator_layers)
gan.train(X, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.610071063041687, generator loss: 0.3254099488258362
Epoch: 1, discriminator loss: 1.5549211502075195, generator loss: 0.3621979355812073
Epoch: 2, discriminator loss: 1.5159568786621094, generator loss: 0.3945510983467102
Epoch: 3, discriminator loss: 1.4824819564819336, generator loss: 0.42640963196754456
Epoch: 4, discriminator loss: 1.4577782154083252, generator loss: 0.4583534300327301
Epoch: 5, discriminator loss: 1.4371135234832764, generator loss: 0.48945292830467224
Epoch: 6, discriminator loss: 1.4213262796401978, generator loss: 0.5113657712936401
Epoch: 7, discriminator loss: 1.4105515480041504, generator loss: 0.5527219176292419
Epoch: 8, discriminator loss: 1.4058856964111328, generator loss: 0.5661323070526123
Epoch: 9, discriminator loss: 1.3885047435760498, generator loss: 0.5910617709159851
Epoch: 10, discriminator loss: 1.404038906097412, generator loss: 0.6071320176124573
Epoch: 11, discriminator loss: 1.408020257949829, generator loss

In [4]:
discriminator_layers=[(7, None), (8, tf.nn.tanh), (3, tf.nn.softmax), (1, None)]
generator_layers=[(9, None), (5, tf.nn.tanh), (4, None)]
cgan = CGAN(discriminator_layers, generator_layers)
cgan.train(X, y, nb_epoch=50, batch_size=5)

Epoch: 0, discriminator loss: 1.4867181777954102, generator loss: 0.42529553174972534
Epoch: 1, discriminator loss: 1.4550148248672485, generator loss: 0.4636271595954895
Epoch: 2, discriminator loss: 1.4329770803451538, generator loss: 0.4982304573059082
Epoch: 3, discriminator loss: 1.4148117303848267, generator loss: 0.5315525531768799
Epoch: 4, discriminator loss: 1.403078317642212, generator loss: 0.5662804841995239
Epoch: 5, discriminator loss: 1.3884347677230835, generator loss: 0.6006386280059814
Epoch: 6, discriminator loss: 1.3834660053253174, generator loss: 0.6324666142463684
Epoch: 7, discriminator loss: 1.3579564094543457, generator loss: 0.6523280143737793
Epoch: 8, discriminator loss: 1.372119665145874, generator loss: 0.6691479682922363
Epoch: 9, discriminator loss: 1.2960481643676758, generator loss: 0.6779193878173828
Epoch: 10, discriminator loss: 1.3683972358703613, generator loss: 0.7042626738548279
Epoch: 11, discriminator loss: 1.3918673992156982, generator loss